In [5]:
# from baza_mip import mip_session, sessionmaker
# from baza_mip.models import Procesy_Przydzielone, Procesy
from sqlalchemy import create_engine, text
from Plan_Pracy.plan_pracy import kj_paczki

import pandas as pd


gen_engine = create_engine("sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/GENERATORY.db", echo=False)



In [9]:
with gen_engine.begin() as conn:
    stmt = conn.execute(text("SELECT * FROM ZAM_PIANKI WHERE nr_partii = '33/03'"))

stmt.all()

[(429, 2432, '16.133.10000.60', 'REVERSO', '5_24', 'REVERSO 1', 55, 'PV', 'V', 'V', 'P', '24/0794', '24/0792', None, None, None, None, None, None, None, None, None, None, None, 'PIANPOL 23_24,VITA 11_24', '33/03', None, None),
 (430, 2432, '16.133.15000.60', 'REVERSO', '5_24', 'REVERSO 1,5', 15, 'PV', 'V', 'V', 'P', '24/0794', '24/0792', None, None, None, None, None, None, None, None, None, None, None, 'PIANPOL 23_24,VITA 11_24', '33/03', None, None),
 (431, 2432, '16.133.20000.60', 'REVERSO', '5_24', 'REVERSO 2', 20, 'PV', 'V', 'V', 'P', '24/0794', '24/0792', None, None, None, None, None, None, None, None, None, None, None, 'PIANPOL 23_24,VITA 11_24', '33/03', None, None),
 (432, 2432, '16.133.63000.60', 'REVERSO', '5_24', 'REVERSO L', 30, 'PV', 'V', 'V', 'P', '24/0794', '24/0792', None, None, None, None, None, None, None, None, None, None, None, 'PIANPOL 23_24,VITA 11_24', '33/03', None, None),
 (433, 2432, '16.133.15031.60', 'REVERSO', '5_24', 'REVERSO T1,5', 50, 'PV', 'V', 'V', 'P'

In [4]:
def dodaj_KJ(zamowione_paczki):
    czas_calkowity = 0
    kajoty = list()
    for row in zamowione_paczki:
        # print(row[0], row[1], row[2], row[3], row[4], row[5])
        
        kajoty.append([f"KJ PARTII {row[5]}: {row[1]} - {row[2]}| NR_DOS {row[3]}", kj_paczki(row[2])*10])
        
        print(f"KJ PARTII {row[5]}: {row[1]} - {row[2]}| NR_DOS {row[3]},", kj_paczki(row[2])*10)
        czas_calkowity += kj_paczki(row[2])*10
    
    print("Całkowity czas KJ:",czas_calkowity)
    return kajoty

def dodaj_OWATY_ROZKROJ(owaty):

    with gen_engine.begin() as conn:
        owaty_mb = conn.execute(text("select * from OWATY")).fetchall()
    mb_owaty = pd.DataFrame(owaty_mb)

    czas_calkowity = 0
    rozkroj_owat = list()
    for row in owaty:
        # print(row[0], row[1], row[2], row[3], row[4])
        czas_ciecia = int(mb_owaty[mb_owaty.OPIS == row[1]].ZUZYCIE.sum() * 1.5)
        czas_calkowity += czas_ciecia
        rozkroj_owat.append([f"WYCINANIE OWAT NR PARTII {row[5]}: {row[1]} - {row[2]}szt| KOMPLETACJA {row[0]}", czas_ciecia])
        print(f"WYCINANIE OWAT NR PARTII {row[5]}: {row[1]} - {row[2]}szt| KOMPLETACJA {row[0]}", czas_ciecia)
    
    print("Całkowity czas ciecia:",czas_calkowity)
    return rozkroj_owat

def dodaj_pakowanie(zamowione_paczki):
    czas_calkowity = 0
    pakowanie = list()
    for row in zamowione_paczki:
        # print(row[0], row[1], row[2], row[3], row[4], row[5])
        
        print(f"PAKOWANIE PARTII {row[5]}: {row[1]} - {row[2]}| KOMPLETACJA {row[0]},", row[2])
        czas_calkowity += row[2]
        pakowanie.append([f"PAKOWANIE PARTII {row[5]}: {row[1]} - {row[2]}| KOMPLETACJA {row[0]}", row[2]])
    
    print("Całkowity czas PAKOWANIA:",czas_calkowity)

    return pakowanie

def wycinanie_memory(memory, model):
    czas_calkowity = 0
  
    for row in memory:
        # print(row[0], row[1], row[2], row[3], row[4], row[5])        
        print(f"WYCINANIE I SKOSOWANIE MEMORY PARTII {row[5]}: {row[1]} - {row[2]}| KOMPLETACJA {row[0]},", row[2])
        czas_calkowity += row[2]
        
    
    print("Całkowity czas WYCINANIA:",czas_calkowity*2)

    return [[f"WYCINANIE I SKOSOWANIE MEMORY PARTII {row[5]}: {model} KOMPLETACJA {row[0]}", czas_calkowity*2 ]]

In [64]:
#dostawy

nr_samochodu = "PIANPOL 26_24"

with gen_engine.begin() as conn:
    suma_paczek = conn.execute(text(f"select SUM(ILE_ZAMOWIONE) from ZAM_PIANKI where nr_SAMOCHODU LIKE '%{nr_samochodu}%'")).fetchall()[0][0]
    zamowione_paczki = conn.execute(text(f"select NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, ZAM1, ZAM2, NR_PARTII from ZAM_PIANKI where nr_SAMOCHODU LIKE '%{nr_samochodu}%'")).fetchall()
    owaty = conn.execute(text(f"select NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, ZAM1, ZAM2, NR_PARTII from ZAM_PIANKI where nr_SAMOCHODU LIKE '%{nr_samochodu}%'")).fetchall()
    memory = conn.execute(text(f"select NR_KOMPLETACJI, OPIS, ILE_ZAMOWIONE, ZAM1, ZAM2, NR_PARTII from ZAM_PIANKI where nr_SAMOCHODU LIKE '%{nr_samochodu}%' AND LENIWA == 'O'")).fetchall()
    


In [65]:
memory

[]

In [66]:
suma_paczek

320

In [67]:
zamowione_paczki

[('2_23', 'DUO 2,5', 5, '24/0824', '', '34/01'),
 ('2_23', 'DUO LH][', 10, '24/0824', '', '34/01'),
 ('2_23', 'DUO ][', 40, '24/0824', '', '34/01'),
 ('2_23', 'DUO P82x82', 5, '24/0824', '', '34/01'),
 ('2_23', 'DUO P62x62', 5, '24/0824', '', '34/01'),
 ('2_23', 'DUO P102x82', 5, '24/0824', '', '34/01'),
 ('3_24', 'ELIXIR 3,5', 40, '24/0824', '', '34/01'),
 ('3_24', 'ELIXIR 4', 40, '24/0824', '', '34/01'),
 ('3_24', 'ELIXIR P120x98', 10, '24/0824', '', '34/01'),
 ('3_24', 'ELIXIR P100x98', 10, '24/0824', '', '34/01'),
 ('3_24', 'HUDSON [L', 10, '24/0824', '', '34/01'),
 ('4_24', 'MAXWELL 4', 10, '24/0824', '', '34/01'),
 ('4_24', 'MAXWELL ][', 50, '24/0824', '', '34/01'),
 ('3_24', 'ONYX 2,5', 10, '24/0824', '', '34/01'),
 ('3_24', 'ONYX 3,5', 30, '24/0824', '', '34/01'),
 ('4_24', 'SAMOA 1z', 30, '24/0824', '', '34/01'),
 ('4_24', 'SAMOA P100x100', 5, '24/0824', '', '34/01'),
 ('5_24', 'DIVA N', 5, '24/0824', '', '34/01')]

In [68]:
pakowanie = dodaj_pakowanie(zamowione_paczki)

PAKOWANIE PARTII 34/01: DUO 2,5 - 5| KOMPLETACJA 2_23, 5
PAKOWANIE PARTII 34/01: DUO LH][ - 10| KOMPLETACJA 2_23, 10
PAKOWANIE PARTII 34/01: DUO ][ - 40| KOMPLETACJA 2_23, 40
PAKOWANIE PARTII 34/01: DUO P82x82 - 5| KOMPLETACJA 2_23, 5
PAKOWANIE PARTII 34/01: DUO P62x62 - 5| KOMPLETACJA 2_23, 5
PAKOWANIE PARTII 34/01: DUO P102x82 - 5| KOMPLETACJA 2_23, 5
PAKOWANIE PARTII 34/01: ELIXIR 3,5 - 40| KOMPLETACJA 3_24, 40
PAKOWANIE PARTII 34/01: ELIXIR 4 - 40| KOMPLETACJA 3_24, 40
PAKOWANIE PARTII 34/01: ELIXIR P120x98 - 10| KOMPLETACJA 3_24, 10
PAKOWANIE PARTII 34/01: ELIXIR P100x98 - 10| KOMPLETACJA 3_24, 10
PAKOWANIE PARTII 34/01: HUDSON [L - 10| KOMPLETACJA 3_24, 10
PAKOWANIE PARTII 34/01: MAXWELL 4 - 10| KOMPLETACJA 4_24, 10
PAKOWANIE PARTII 34/01: MAXWELL ][ - 50| KOMPLETACJA 4_24, 50
PAKOWANIE PARTII 34/01: ONYX 2,5 - 10| KOMPLETACJA 3_24, 10
PAKOWANIE PARTII 34/01: ONYX 3,5 - 30| KOMPLETACJA 3_24, 30
PAKOWANIE PARTII 34/01: SAMOA 1z - 30| KOMPLETACJA 4_24, 30
PAKOWANIE PARTII 34/01: SA

In [69]:
kjoty = dodaj_KJ(zamowione_paczki)

KJ PARTII 34/01: DUO 2,5 - 5| NR_DOS 24/0824, 10
KJ PARTII 34/01: DUO LH][ - 10| NR_DOS 24/0824, 10
KJ PARTII 34/01: DUO ][ - 40| NR_DOS 24/0824, 40
KJ PARTII 34/01: DUO P82x82 - 5| NR_DOS 24/0824, 10
KJ PARTII 34/01: DUO P62x62 - 5| NR_DOS 24/0824, 10
KJ PARTII 34/01: DUO P102x82 - 5| NR_DOS 24/0824, 10
KJ PARTII 34/01: ELIXIR 3,5 - 40| NR_DOS 24/0824, 40
KJ PARTII 34/01: ELIXIR 4 - 40| NR_DOS 24/0824, 40
KJ PARTII 34/01: ELIXIR P120x98 - 10| NR_DOS 24/0824, 10
KJ PARTII 34/01: ELIXIR P100x98 - 10| NR_DOS 24/0824, 10
KJ PARTII 34/01: HUDSON [L - 10| NR_DOS 24/0824, 10
KJ PARTII 34/01: MAXWELL 4 - 10| NR_DOS 24/0824, 10
KJ PARTII 34/01: MAXWELL ][ - 50| NR_DOS 24/0824, 50
KJ PARTII 34/01: ONYX 2,5 - 10| NR_DOS 24/0824, 10
KJ PARTII 34/01: ONYX 3,5 - 30| NR_DOS 24/0824, 30
KJ PARTII 34/01: SAMOA 1z - 30| NR_DOS 24/0824, 30
KJ PARTII 34/01: SAMOA P100x100 - 5| NR_DOS 24/0824, 10
KJ PARTII 34/01: DIVA N - 5| NR_DOS 24/0824, 10
Całkowity czas KJ: 350


In [70]:
wycinanie = wycinanie_memory(memory, "AVANT")
wycinanie

Całkowity czas WYCINANIA: 0


UnboundLocalError: cannot access local variable 'row' where it is not associated with a value

In [71]:
owaty = dodaj_OWATY_ROZKROJ(zamowione_paczki)

WYCINANIE OWAT NR PARTII 34/01: DUO 2,5 - 5szt| KOMPLETACJA 2_23 7
WYCINANIE OWAT NR PARTII 34/01: DUO LH][ - 10szt| KOMPLETACJA 2_23 12
WYCINANIE OWAT NR PARTII 34/01: DUO ][ - 40szt| KOMPLETACJA 2_23 1
WYCINANIE OWAT NR PARTII 34/01: DUO P82x82 - 5szt| KOMPLETACJA 2_23 2
WYCINANIE OWAT NR PARTII 34/01: DUO P62x62 - 5szt| KOMPLETACJA 2_23 2
WYCINANIE OWAT NR PARTII 34/01: DUO P102x82 - 5szt| KOMPLETACJA 2_23 3
WYCINANIE OWAT NR PARTII 34/01: ELIXIR 3,5 - 40szt| KOMPLETACJA 3_24 6
WYCINANIE OWAT NR PARTII 34/01: ELIXIR 4 - 40szt| KOMPLETACJA 3_24 7
WYCINANIE OWAT NR PARTII 34/01: ELIXIR P120x98 - 10szt| KOMPLETACJA 3_24 2
WYCINANIE OWAT NR PARTII 34/01: ELIXIR P100x98 - 10szt| KOMPLETACJA 3_24 2
WYCINANIE OWAT NR PARTII 34/01: HUDSON [L - 10szt| KOMPLETACJA 3_24 9
WYCINANIE OWAT NR PARTII 34/01: MAXWELL 4 - 10szt| KOMPLETACJA 4_24 8
WYCINANIE OWAT NR PARTII 34/01: MAXWELL ][ - 50szt| KOMPLETACJA 4_24 1
WYCINANIE OWAT NR PARTII 34/01: ONYX 2,5 - 10szt| KOMPLETACJA 3_24 7
WYCINANIE OWAT 

In [72]:
nr_partii = zamowione_paczki[0][-1]
nr_partii

'34/01'

In [73]:
##eliza 5
##adamr 4
##piotrl 8
planowany_dzien_rozpoczecia = "2024-09-20"


mip_session.add(Procesy_Przydzielone(4, 5, 1, f"DOSTAWA PARTII {nr_partii}: NR SAM {nr_samochodu}", planowany_dzien_rozpoczecia, suma_paczek))
# mip_session.add(Procesy_Przydzielone(4, 5, 1, f"DOSTAWA PARTII 33/03: NR SAM VITA 11_24", planowany_dzien_rozpoczecia, 570))

for kj in kjoty:
    mip_session.add(Procesy_Przydzielone(4, 5, 2, kj[0], planowany_dzien_rozpoczecia, kj[1]))

for p in pakowanie:
    mip_session.add(Procesy_Przydzielone(4, 5, 3, p[0], planowany_dzien_rozpoczecia, p[1]))

# mip_session.add(Procesy_Przydzielone(4, 5, 11, "DOSTAWA OWAT Z:50, N:20, C:80", "2024-08-05", 150*9))

# for w in wycinanie:
#     mip_session.add(Procesy_Przydzielone(4, 5, 10, w[0], planowany_dzien_rozpoczecia, w[1]))

for o in owaty:
    mip_session.add(Procesy_Przydzielone(4, 5, 9, o[0], planowany_dzien_rozpoczecia, o[1]))


# mip_session.commit()

In [ ]:
prtia = '34/01'
mip_session.execute(text(f"""SELECT uid, kid, proces, nazwa_procesu, data_utworzenia, planowany_dzien_rozpoczecia, priorytet, status, aktywna, uwagi_kier, preferowany_czas_wykonania 
                         FROM baza_mip.procesy_przydzielone where data_utworzenia = '2024-08-22'""")).all()